### Memory Analysis Volatility Playbook  

![Static Badge](https://img.shields.io/badge/john_coreano-blue)
![Static Badge](https://img.shields.io/badge/Memory_Analysis_Playbook-yellow)
![Static Badge](https://img.shields.io/badge/CTF-red)

This playbook should give you a good start on analyzing memory. It is handy when working on CTF's and for learning and nailing down a good analytical process of your own.  

This playbook assumes that:
- you have a working knowledge on memory capture
- you have the ability to modify commands to suit your needs
- you are comfortable navigating via a Terminal or Command Line
- that you already have Volatility2 installed    

This is not meant to be all inclusive but rather a guide to get you going and to provide a structure to memory analysis. If you have any suggestions to make this user fiendly, let me know!

#### Analysis Techniques

- Expect large quantities of data and a lot of noise. 
- Memory is heavily fragmented.
- Don’t jump to conclusions!
- Create a timeline.
- Verify integrity of the image by verifying the hash.
- Try different options.
- Use the *help* command on plugins to see syntax and what they can do.

#### Volatility 2

Syntax: `volatility -f <FILENAME> --profile=PROFILE <PLUGIN>`  
** *--profile may not always be needed, especially if working in lab environments*

##### 1) <u>Verify integrity of the image by verifying its hash.</u>
##### `md5sum FILENAME`
##### `sha256sum FILENAME`

##### 2) <u>Identify information about the image.</u>
##### Plugins to use: imageinfo, ident, kdbgscan, datetime  
##### Example: `volatility -f IMAGE imageinfo`

##### ** At this point begin creating your timeline with the image creation date and time. **

##### 3) <u>Identify the processes running at time of memory capture.</u>  
##### Plugins to use: pslist, pstree, pstree -v
##### Example: `volatility -f IMAGE pstree -v` 
##### Look for suspicious processes. The more familiar you are with processes that should be running, the easier it will be to spot anything out of the ordinary. Place any suspicious processes into your timeline. Note the Process Name, Process ID (PID), the Parent Process ID (PPID), and the time. 

##### 4) <u>List the DLL's used by processes.</u>  
##### Plugins to use: dlllist -p PID
##### Example: `volatility -f IMAGE dlllist -p PID`
##### This will list all the Dynamically Linked Libraries (DLL) that a process used. Look for anything that looks suspicious. DLL's should normally reside in the C:\WINDOWS\system32\ folder.

##### 5) <u>List any files accessed by a process.</u>
##### Plugins to use: files -p PID
##### Example: `volatility -f IMAGE files -p PID`

##### 6) <u>Show which user Security Identifier's (SIDS) launched a process.</u>  
##### Plugins to use: getsids
##### Example: `volatility -f IMAGE getsids`

##### 7) <u>List network connections and any sockets created by processes at the time the image was taken.</u>  
##### Plugins to use: sockets, sockscan, sockscan2, connections, connscan, connscan2
##### Example: `volatility -f IMAGE sockscan`
##### Try them all out and see what works for you and what information you can get from them. Put anything you have a timestamp for in your timeline.

##### 8) <u>Analyze command line arguments.</u>
##### Plugins to use: cmdscan, consoles, cmdline
##### Example: `volatility -f IMAGE consoles` or `volatility -f IMAGE consoles -p PID`

#### Advanced options

##### 9) <u>Analyze the registry.</u>
##### Create a timeline of the registry.
##### - Identify the memory offset of the registry hive: `volatility -f IMAGE hivescan`
##### Note the hex value of the offset from the first line for use in the next command.
##### - Dump the registry to a CSV file. `volatility hivedump -f IMAGE -o HEXOFFSET -v`
##### Now we will clean up the CSV file and create a timeline document.
##### - `sort -n *.csv > registry_timeline.csv`
##### List registry keys opened by a PID.  
##### - `volatility -f IMAGE regobjkeys -p PID`
##### List the registry hives using the HEXOFFSET.
##### - `volatility hivelist -f IMAGE -o HEXOFFSET` 
##### Print out the value of a specific registry key using the ADDRESS obtained from the hive list.
##### - `volatility printkey -f IMAGE -o ADDRESS 'HIVENAME'` 

##### 10) <u>Check for passwords or any secret information.</u>  
##### Plugins to use: hashdump, lsadump, cachedump, cryptoscan, suspicious

##### 11) <u>Look for signs of malware.</u>
##### Plugins to use: malfind 
##### Example: `volatility -f IMAGE malfind` or `volatility -f IMAGE malfind -p PID`

#### Support  